In [1]:
import os
import numpy as np
import cv2
from skimage.morphology import binary_dilation, disk, dilation
from scipy.ndimage import distance_transform_edt
from edit_annotation_tiles import edit_annotations_tiles
from load_annotation_data import load_annotation_data

In [2]:
# Pre - inputs
pth = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model'
pthDL = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\04_19_2024'
pthim_ann = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\5x'
classcheck = 0
datafile = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\04_19_2024\net.pkl'

In [3]:
# Inputs 
import pickle
with open(datafile, 'rb') as f:
    data = pickle.load(f)
nblack = data['nblack']
sxy = data['sxy']
ctlist0, numann0 = load_annotation_data(pthDL, pth, pthim_ann,classcheck)
numann0 = np.array(numann0)  # Convert numann0 to a NumPy array

numann = numann0.copy()
percann = np.double(numann0 > 0)
percann = np.dstack((percann, percann))
percann0 = percann.copy()
stile = None 
nbg = None

outpth = r'training'


 
Importing annotation data...
Image 1 of 2: SG_013_0061
 1. of 4. Importing annotation data from xml file
 Creating file...
  Existing TA loaded
 2. of 4. Interpolating annotated regions and saving mask image
 4. of 4. Creating bounding box tiles of all annotations
 Finished image in 108 seconds.
Image 2 of 2: SG_014_0016
 1. of 4. Importing annotation data from xml file
 Creating file...
  Existing TA loaded
 2. of 4. Interpolating annotated regions and saving mask image
 4. of 4. Creating bounding box tiles of all annotations
 Finished image in 460 seconds.


In [7]:
if nbg is None:
        nbg = 0
if stile is None:
    stile = 10000
stile += 200
kpall = 1


In [8]:
# Define folder locations
outpthim = os.path.join(pthDL, outpth, 'im/')
outpthlabel = os.path.join(pthDL, outpth, 'label/')
outpthbg = os.path.join(pthDL, outpth, 'big_tiles/')
os.makedirs(outpthim, exist_ok=True)
os.makedirs(outpthlabel, exist_ok=True)
os.makedirs(outpthbg, exist_ok=True)

In [9]:
imlistck = [f for f in os.listdir(outpthim) if f.endswith('.tif')]
nm0 = len(imlistck) + 1

In [10]:
# Create very large blank images - NO IDEA WHAT THESE VARIABLES ARE
imH = np.full((stile, stile, 3), nbg, dtype=np.double) #create an array with the specified dimensions with the value of ngb
imT = np.zeros((stile, stile)) #blank mask
nL = imT.size # size of the big tile
ct = np.zeros(numann.shape[1]) # list (size of how many annotations you have)
sf = np.sum(ct) / nL # how full is the big tile

In [13]:
count = 1
tcount = 0 #is not one because of python indexing starts at 0
cutoff = 0.55 # Cut off for how full the big tile is (55% full)
rsf = 5
type0 = 1
h = np.ones((51, 51))
h[25, 25] = 0
h = distance_transform_edt(h) < 26

In [14]:
#__________ This should be the iteration of a  while loop__________
# choose one of each class in order in a loop
if count % 10 == 1:
    type_ = tcount
    tcount = (tcount % len(ct)) + 1
# choose a tile containing the least prevalent class
else:
    tmp = np.sum(ct, axis=1)
    tmp[type0] = np.max(tmp) #
    type_ = np.argmin(tmp)
    
num = np.where(numann[:, type_] > 0)[0]

if len(num) == 0:
    numann[:, type_] = numann0[:, type_]
    num = np.where(numann[:, type_] > 0)[0]
num = np.random.choice(num, size=1, replace=False)


KeyboardInterrupt: 

In [108]:
# load annotation and mask
tile_name = ctlist0[num[0]]
print(tile_name)
pthim = os.path.join(pthDL, tile_name)

UsageError: Line magic function `%` not found.


In [8]:
%reset

In [109]:
with open(datafile, 'rb') as f:
    data = pickle.load(f)
pth = data['pthim']
print(pth)

\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\5x
